# Лабораторная работа №6

Селивёрстов Д.С. М8О-401Б-21

# Выбор датасета

Выбранный датасет - "CIFAR-10".

*Описание*:
- 60,000 изображений размером 32x32 пикселя (3 канала).
- 10 классов: `airplane`, `automobile`, `bird`, `cat`, `deer`, `dog`, `frog`, `horse`, `ship`, `truck`.
- Разделён на 50,000 обучающих и 10,000 тестовых примеров.


*Задача классификации*: классификация объекта на изображении в один из 10 классов.

# Метрики

- Accuracy (доля правильных предсказаний): подходит, т.к. классы сбалансированы (примерно по 6,000 изображений на класс), даёт быструю общую оценку качества модели.

- F1-Score: показывает качество предсказаний по всем классам, особенно при анализе ошибок.


In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

### Импорт библиотек

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score
from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используемое устройство:", device)

Используемое устройство: cuda


### Преобразования и загрузка CIFAR-10

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = trainset.classes

### Обучение сверточной модели (ResNet18)

In [ ]:
from torchvision.models import resnet18

model_resnet = resnet18(pretrained=True)
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, 10)
model_resnet = model_resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet.parameters(), lr=0.001)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Функция обучения

In [ ]:
def train_model(model, trainloader, criterion, optimizer, epochs=3):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in tqdm(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"\nЭпоха {epoch + 1}, Потери: {running_loss / len(trainloader):.4f}")

Обучение ResNet

In [ ]:
train_model(model_resnet, trainloader, criterion, optimizer)

100%|██████████| 782/782 [02:36<00:00,  5.00it/s]



Эпоха 1, Потери: 0.2430


100%|██████████| 782/782 [02:37<00:00,  4.97it/s]



Эпоха 2, Потери: 0.2417


100%|██████████| 782/782 [02:36<00:00,  5.01it/s]


Эпоха 3, Потери: 0.2419


### Обучение трансформера (deit)



Обучение deit

In [ ]:
!pip install timm

In [ ]:
import timm

In [ ]:
model = timm.create_model('deit_tiny_patch16_224', pretrained=True, num_classes=10)
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

def train_model(model, trainloader, epochs=3):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in tqdm(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"\nЭпоха {epoch + 1}, Потери: {running_loss / len(trainloader):.4f}")

In [ ]:
train_model(model, trainloader)

100%|██████████| 782/782 [03:05<00:00,  4.21it/s]



Эпоха 1, Потери: 0.3291


100%|██████████| 782/782 [03:05<00:00,  4.21it/s]



Эпоха 2, Потери: 0.2056


100%|██████████| 782/782 [03:05<00:00,  4.21it/s]


Эпоха 3, Потери: 0.1587


### Оценка по метрикам

In [ ]:
def evaluate_model_resnet(model, testloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)

            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    acc = MulticlassAccuracy(num_classes=10, average='macro')(all_preds, all_labels)
    f1 = MulticlassF1Score(num_classes=10, average='macro')(all_preds, all_labels)
    print(f"Accuracy: {acc:.4f}, Macro F1-score: {f1:.4f}")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate_model_deit(model, testloader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    print(f"\nAccuracy: {acc:.4f}")
    print(f"F1 Score (weighted): {f1:.4f}")

Оценка ResNet

In [ ]:
evaluate_model_resnet(model_resnet, testloader)

Accuracy: 0.8763, Macro F1-score: 0.8766


Оценка ViT

In [ ]:
evaluate_model_deit(model, testloader)


Accuracy: 0.9223
F1 Score (weighted): 0.9218


### Улучшение бейзлайна

Добавим аугментации данных при обучении моделей.

- RandomHorizontalFlip: поворачивает изображение по горизонтали, что помогает избежать переобучения.
- RandomCrop: обрезка с последующим ресайзом усиливает устойчивость модели.
- ColorJitter: случайно меняет яркость, контраст и насыщенность.
- RandomRotation: немного поворачивает изображения, имитируя реальную вариацию.

Обновим трансформации

Обучение Resnet

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet.parameters(), lr=0.001)

def train_model(model, trainloader, criterion, optimizer, epochs=3):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in tqdm(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"\nЭпоха {epoch + 1}, Потери: {running_loss / len(trainloader):.4f}")

train_model(model_resnet, trainloader_aug, criterion, optimizer)

100%|██████████| 782/782 [05:21<00:00,  2.43it/s]



Эпоха 1, Потери: 0.4117


100%|██████████| 782/782 [05:20<00:00,  2.44it/s]



Эпоха 2, Потери: 0.3303


100%|██████████| 782/782 [05:18<00:00,  2.45it/s]


Эпоха 3, Потери: 0.2800


Обучение deit

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

train_model(model, trainloader_aug, criterion, optimizer)

100%|██████████| 782/782 [05:45<00:00,  2.26it/s]



Эпоха 1, Потери: 0.2446


100%|██████████| 782/782 [05:46<00:00,  2.25it/s]



Эпоха 2, Потери: 0.2089


100%|██████████| 782/782 [05:45<00:00,  2.26it/s]


Эпоха 3, Потери: 0.1825


Оценка ResNet

In [ ]:
evaluate_model_resnet(model_resnet, testloader)

Accuracy: 0.8964, Macro F1-score: 0.8972


Оценка ViT

In [ ]:
evaluate_model_deit(model, testloader)


Accuracy: 0.9222
F1 Score (weighted): 0.9224


### Сравнение результатов

| Модель     | Accuracy (до) | F1 (до)    | Accuracy (после) | F1 (после)  |
|------------|---------------|------------|------------------|-------------|
| ResNet18   | 0.8763        | 0.8766     | **0.8964**       | **0.8972**  |
| DeiT-tiny  | 0.9223        | 0.9218     | **0.9222**       | **0.9224**  |


### Вывод

Аугментации данных — простой и эффективный способ улучшить качество ResNet18.

Они помогают предотвратить переобучение и обучить более устойчивую модель на относительно небольшом датасете, как CIFAR-10.

DeiT уже показывает высокий уровень качества без аугментаций.

Его архитектура и предобученность дают сильный старт. Аугментации дали лишь микроскопическое улучшение.

Вывод:

- Для простых моделей, таких как ResNet18, улучшения бейзлайна через аугментации — высокоэффективны.

- Для более мощных моделей, таких как DeiT, дальнейшее улучшение стоит искать в более продвинутом тюнинге: подбор learning rate, scheduler, optimizer, увеличение числа эпох, возможно fine-tuning последних слоёв, если обучать на своих данных.

## Имплементация алгоритма машинного обучения

Импорты и подготовка

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Загрузка и подготовка CIFAR-10

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

Свёрточная модель (простая CNN)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32x16x16

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 64x8x8
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        return self.fc(x)

model_cnn = SimpleCNN().to(device)


Обучающая функция

In [ ]:
def train_model(model, train_loader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")


Функция оценки модели

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    preds, labels_all = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            preds.extend(predicted.cpu().numpy())
            labels_all.extend(labels.numpy())

    acc = accuracy_score(labels_all, preds)
    f1 = f1_score(labels_all, preds, average='macro')
    print(f"Accuracy: {acc:.4f}, Macro F1-score: {f1:.4f}")
    return acc, f1


Vision Transformer

In [ ]:
import math

class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=3, patch_size=4, emb_size=128, img_size=32):
        super().__init__()
        self.patch_size = patch_size
        self.emb_size = emb_size
        self.n_patches = (img_size // patch_size) ** 2
        self.proj = nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)  # (B, emb_size, H/patch, W/patch)
        x = x.flatten(2)  # (B, emb_size, n_patches)
        x = x.transpose(1, 2)  # (B, n_patches, emb_size)
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, emb_size=128, num_heads=4, dropout=0.1, forward_expansion=4):
        super().__init__()
        self.layernorm1 = nn.LayerNorm(emb_size)
        self.attn = nn.MultiheadAttention(emb_size, num_heads, dropout=dropout, batch_first=True)
        self.layernorm2 = nn.LayerNorm(emb_size)

        self.mlp = nn.Sequential(
            nn.Linear(emb_size, emb_size * forward_expansion),
            nn.GELU(),
            nn.Linear(emb_size * forward_expansion, emb_size),
        )

    def forward(self, x):
        x_attn = self.attn(x, x, x, need_weights=False)[0]
        x = x + x_attn
        x = self.layernorm1(x)

        x_mlp = self.mlp(x)
        x = x + x_mlp
        x = self.layernorm2(x)
        return x

class SimpleViT(nn.Module):
    def __init__(self, img_size=32, patch_size=4, in_channels=3, emb_size=128, num_classes=10, depth=6):
        super().__init__()
        self.patch_embed = PatchEmbedding(in_channels, patch_size, emb_size, img_size)
        n_patches = (img_size // patch_size) ** 2

        self.cls_token = nn.Parameter(torch.randn(1, 1, emb_size))
        self.pos_embed = nn.Parameter(torch.randn(1, n_patches + 1, emb_size))

        self.transformer = nn.Sequential(*[
            TransformerEncoder(emb_size) for _ in range(depth)
        ])

        self.norm = nn.LayerNorm(emb_size)
        self.head = nn.Linear(emb_size, num_classes)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)  # (B, n_patches, emb_size)
        cls_tokens = self.cls_token.expand(B, -1, -1)  # (B, 1, emb_size)
        x = torch.cat([cls_tokens, x], dim=1)  # (B, n_patches+1, emb_size)
        x = x + self.pos_embed

        x = self.transformer(x)
        x = self.norm(x[:, 0])  # Use cls token
        return self.head(x)

model_vit = SimpleViT().to(device)


### Обучение и оценка

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_cnn = optim.Adam(model_cnn.parameters(), lr=0.001)

train_model(model_cnn, train_loader, optimizer_cnn, criterion, epochs=10)
acc_cnn, f1_cnn = evaluate_model(model_cnn, test_loader)

Epoch 1/10, Loss: 1.3590
Epoch 2/10, Loss: 0.9792
Epoch 3/10, Loss: 0.8207
Epoch 4/10, Loss: 0.7047
Epoch 5/10, Loss: 0.6053
Epoch 6/10, Loss: 0.5161
Epoch 7/10, Loss: 0.4225
Epoch 8/10, Loss: 0.3461
Epoch 9/10, Loss: 0.2717
Epoch 10/10, Loss: 0.2131
Accuracy: 0.7145, Macro F1-score: 0.7152


In [ ]:
optimizer_vit = optim.Adam(model_vit.parameters(), lr=0.001)

train_model(model_vit, train_loader, optimizer_vit, criterion, epochs=10)
acc_vit, f1_vit = evaluate_model(model_vit, test_loader)


Epoch 1/10, Loss: 1.7826
Epoch 2/10, Loss: 1.5027
Epoch 3/10, Loss: 1.3796
Epoch 4/10, Loss: 1.2858
Epoch 5/10, Loss: 1.2208
Epoch 6/10, Loss: 1.1552
Epoch 7/10, Loss: 1.1012
Epoch 8/10, Loss: 1.0544
Epoch 9/10, Loss: 0.9977
Epoch 10/10, Loss: 0.9524
Accuracy: 0.6136, Macro F1-score: 0.6108


### Сравнение и выводы

| Модель                      | Accuracy | F1-score (macro/weighted) |
|----------------------------|----------|----------------------------|
| **ResNet18 (базовый)**     | 0.8763   | 0.8766                     |
| **DeiT Tiny (базовый)**    | 0.9223   | 0.9218                     |
| **Собственная CNN**        | 0.7145   | 0.7152                     |
| **Собственный ViT**        | 0.6136   | 0.6108                     |


- DeiT Tiny остаётся самой точной моделью.

- Собственные реализации ViT и CNN работают хуже предобученных моделей. Это ожидаемо:

  1. Упрощённый ViT страдает из-за меньшей глубины и отсутствия предобученных весов.

  2. Простая CNN не может конкурировать с глубокими архитектурами без серьёзной доработки.

- Эти результаты подчёркивают важность предварительного обучения и архитектурной глубины для современных моделей — особенно для трансформеров, которым требуется много данных и вычислений.

### Улучшение бейзлайна

In [ ]:
import torchvision.transforms as transforms

# Аугментации для обучения
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandAugment(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Аугментации для валидации/тестов
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=train_transform)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 56 * 56, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=128):
        super().__init__()
        self.patch_dim = patch_size * patch_size * in_channels
        self.n_patches = (img_size // patch_size) ** 2
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)  # (B, embed_dim, H, W)
        x = x.flatten(2)  # (B, embed_dim, N)
        x = x.transpose(1, 2)  # (B, N, embed_dim)
        return x

class SimpleViT(nn.Module):
    def __init__(self, num_classes=10, embed_dim=128, num_heads=4, depth=4):
        super().__init__()
        self.patch_embed = PatchEmbedding(embed_dim=embed_dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.randn(1, 197, embed_dim))  # 196 patches + 1 cls

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=depth)
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x):
        x = self.patch_embed(x)
        cls_tokens = self.cls_token.expand(x.size(0), -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed[:, :x.size(1), :]
        x = self.transformer(x)
        return self.mlp_head(x[:, 0])


In [ ]:
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

def evaluate_model(model, test_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    print(f"Accuracy: {acc:.4f}, Macro F1-score: {f1:.4f}")
    return acc, f1


Обучение и метрики

In [ ]:
# CNN
cnn = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=1e-3)
train_model(cnn, train_loader, criterion, optimizer, epochs=5)
acc_cnn, f1_cnn = evaluate_model(cnn, test_loader)

Epoch 1, Loss: 1.9169
Epoch 2, Loss: 1.6127
Epoch 3, Loss: 1.4981
Epoch 4, Loss: 1.4473
Epoch 5, Loss: 1.3839
Accuracy: 0.5538, Macro F1-score: 0.5501


In [ ]:
# Vision Transformer
vit = SimpleViT().to(device)
optimizer = optim.Adam(vit.parameters(), lr=1e-3)
train_model(vit, train_loader, criterion, optimizer, epochs=5)
acc_vit, f1_vit = evaluate_model(vit, test_loader)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1, Loss: 2.3253
Epoch 2, Loss: 2.3087
Epoch 3, Loss: 2.3060
Epoch 4, Loss: 2.3051
Epoch 5, Loss: 2.3043
Accuracy: 0.1000, Macro F1-score: 0.0182


### Сравнение и выводы

| Модель                   | Accuracy | Macro F1-score |
|--------------------------|----------|----------------|
| ResNet (pretrained)      | 0.8964   | 0.8972         |
| DeiT Tiny (pretrained)   | 0.9222   | 0.9224         |
| Simple CNN (custom)      | 0.5538   | 0.5501         |
| Simple ViT (custom)      | 0.1000   | 0.0182         |

1. Предобученные модели (ResNet и DeiT) с улучшенным бейзлайном (аугментации RandAugment) показали высокие результаты, особенно DeiT (Accuracy > 92%).

2. Собственные реализации моделей, несмотря на применение таких же аугментаций, значительно уступают по качеству:

  - Simple CNN достигла лишь ~55% Accuracy.

  - Simple ViT почти не обучился, его Accuracy ≈ случайному угадыванию.

3. Это объясняется:

  - Отсутствием глубоких архитектур и обилия параметров в самописных моделях.

  - Недостаточной тренировкой (мелкие трансформеры сложны для обучения "с нуля").

  - Отсутствием оптимизаций, применённых в продвинутых моделях (timm, torchvision).

4. Улучшенный бейзлайн критически важен для достижения высоких метрик, но предобученные веса и зрелые архитектуры — основа хорошей производительности.